In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.fft
import scipy.io
from scipy.io import wavfile
from scipy.stats import norm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchsummary import summary
import torchvision
from sklearn.cluster import KMeans



In [ ]:
def pdf(res,b):
    (mu, sigma) = norm.fit(res)
    distr, bins= np.histogram(res, b,density =True)
    y = scipy.stats.norm.pdf( bins, mu, sigma)
    return bins,y   

In [ ]:
def find_noise(Segments):
    #small energy=noisy segments
    Energies=np.log(np.sum(Segments**2,axis=1)).reshape(-1,1)
    kmeans = KMeans(n_clusters=2, random_state=0).fit(Energies)
    labels=kmeans.predict(Energies, sample_weight=None)
    c1=Segments[np.where(labels==0)]
    c2=Segments[np.where(labels==1)]
    if np.mean(Energies[np.where(labels==0)],axis=0)<np.mean(Energies[np.where(labels==1)]):
        Small_energy=c1
    else:
        Small_energy=c2
    
    Noise=Small_energy[np.where(Small_energy.std(axis=1)<=np.mean(Small_energy.std(axis=1)))]
    if Noise.shape[0]>0:
        return Noise
    else:
        return Small_energy
    

In [ ]:
def noise_reduce(sig,sampling_rate,alpha,beta=0.01,time_ms=30,time_s=None):

    if time_s==None:
        time_s=9999
    block_size=int((time_ms*(10**(-3)))*sr)
    j=int((np.floor(sig.shape[0]/block_size)))
    
    Segmented_signal=sig[:j*block_size].reshape(-1,block_size)
    
    if (np.floor(sig.shape[0]%block_size))!=0:
        Lost_signal=sig[j*block_size:]
        last_segment=np.zeros(block_size)
        last_segment[:Lost_signal.shape[0]]=Lost_signal
        last_segment=last_segment.reshape(1,-1)
        Segmented_signal=np.concatenate([Segmented_signal,last_segment],axis=0)
        
    #Find noise
    subsignal=sig[:time_s*sampling_rate]
    
    k=int((np.floor(subsignal.shape[0]/block_size)))
    
    Segmented_subsignal=subsignal[:k*block_size].reshape(-1,block_size)
    
    noise=find_noise(Segmented_subsignal)
    noise_model=np.mean(np.abs(scipy.fft.fft(noise)),axis=0)
    
    y_fft=(scipy.fft.fft(Segmented_signal))
    Y_mag=(np.abs(y_fft))
    Y_angle=np.angle(y_fft)
    X_mag=Y_mag-alpha*noise_model
    #applying T
    if alpha!=1:
        X_mag[np.where(X_mag<beta*Y_mag)]=beta*Y_mag[np.where(X_mag<beta*Y_mag)]
    X=X_mag*np.exp(1.0j*Y_angle)
    x=scipy.fft.ifft(X).real.astype(np.float32)
    x=x.reshape(-1)[:sig.shape[0]]
    return x

In [ ]:
def myError(x,y,l,W):
  return ((1/2)*(x-y)**2).mean()  +(l/2)*W

def kl_divergence(rho, rho_hat):
    rho_hat = torch.mean(torch.sigmoid(rho_hat), 1) # sigmoid because we need the bernouli probability distributions
    rho = torch.tensor([rho] * (rho_hat).shape[0]).to(device)
    return torch.sum(rho * torch.log(rho/rho_hat) + (1 - rho) * torch.log((1 - rho)/(1 - rho_hat)))
# define the sparse loss function

def sparse_loss(r, input):
    leaky=nn.LeakyReLU(0.2)
    values = input
    loss = 0
    model_children = list(model.children())
    for i in range(np.array(model_children).shape[0]):
      values = leaky((model_children[i](values)))
      if i in np.arange(len(model_children))[:-2]:#we need the hidden layers 
        loss += kl_divergence(r, values)
    return loss

def calcW(model):
  model_children = list(model.children())
  loss = 0
  for i in range(np.array(model_children).shape[0]):
    if i in np.arange(len(model_children))[1:-1]:#proposed
        loss += ((abs(model_children[i].weight.data)**2).mean())
  return loss

def clean_signal(signal,model,window_length,device):
  segments=[]
  for k in range(int(((signal).shape[0]/window_length))):
    segment=signal[k*window_length:(k+1)*window_length]
    segments.append(segment)
  
  if signal.shape[0]%window_length!=0:
        last_segment=np.zeros(window_length)
        last_segment[:signal[(k+1)*window_length:].shape[0]]=signal[(k+1)*window_length:]
        segments.append(last_segment)
  segments=torch.tensor(segments,dtype=torch.double).to(device)

  clean=model(segments)

  
    
  return clean.reshape(-1).detach().cpu().numpy()[:signal.shape[0]]


def choose_model(model_old,model_new,validation_error_old,validation_error_new):
    if validation_error_new<validation_error_old or validation_error_new==validation_error_old:
        return model_new,validation_error_new
    else:
        return model_old,validation_error_old

# Model training 

In [ ]:
signals=[]
#j='Nikaia'
#j='Patra'
for j in ['Nikaia']:
    for i in range(1,3000):
        try:
        
            _,mixtureR_corr = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_R_scaled')
            _, mixtureR_clean = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_dn/'+str(i)+'_R_denoised')
        
            _, mixtureL_corr = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_L_scaled')
            _, mixtureL_clean = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_dn/'+str(i)+'_L_denoised')
            signals.append([mixtureR_corr,mixtureR_clean])
            signals.append([mixtureL_corr,mixtureL_clean])
       
        except:
            continue
signals=np.array(signals,dtype=object)

#segment the signals into windows (non overlapping)
segments=[]

window_length=784
for corr,clean in signals:
  for k in range(int(((corr).shape[0]/window_length))):
    
    segment_corr=corr[k*window_length:(k+1)*window_length]
    segment_clean=clean[k*window_length:(k+1)*window_length]
    
    segments.append([segment_corr,segment_clean])
segments=np.array(segments)

In [ ]:
np.random.shuffle(segments)

f=int(segments.shape[0]*0.8)
#split the data to trainset/testset
trainset= segments[:f]
testset= segments[f:]

input_size=window_length
batch_size=64

#loading the data into tensors 
X_train=torch.tensor(trainset,dtype=torch.double)
X_test=torch.tensor(testset,dtype=torch.double)
dl_train=DataLoader(X_train,batch_size,shuffle=True)
dl_test=DataLoader(X_test,batch_size,shuffle=True)

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class AutoEncoder(nn.Module):
    #Works Best
    #2 Layers Double size
    def __init__(self):
        super( AutoEncoder, self).__init__()
        # encoder
        self.enc1 = nn.Linear(input_size,768,bias=True)
        self.enc2 = nn.Linear(in_features=768, out_features=700,bias=True)
        
        # decoder 
        
        self.dec1 = nn.Linear(in_features=700, out_features=768,bias=True)
        self.dec2 = nn.Linear(in_features=768, out_features=input_size,bias=True)
        
    def forward(self, x):
        #paper proposes sigmoid 
        self.leaky=nn.LeakyReLU(0.2)
        # encoding
        x = self.leaky(self.enc1(x))
        x = self.leaky(self.enc2(x))
        
        # decoding
        x = self.leaky(self.dec1(x))
        x = (self.dec2(x))
        
        return x

learning_rate=1e-5
model=AutoEncoder().double().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
Error=[]
Validation=[]
Outputs=[]

In [ ]:
Epochs=900
lamda=1
beta=0
rho=0.05
#when to switch?
delta=1e-1

for epoch in range(Epochs):
  e=[]
  for i in dl_train:
    corrupted_data,clean_data=i[:,0,:],i[:,1,:]
    corrupted_data=corrupted_data.reshape(-1,input_size).to(device)
    clean_data=clean_data.reshape(-1,input_size).to(device)

    #
    s=clean_data.cpu().numpy()
    clean_data[np.where(s.std(axis=1)<delta)]=torch.zeros(clean_data[np.where(s.std(axis=1)<delta)].shape,dtype=torch.float64).to(device)
    
        # ===================forward=====================
       
    output = model(corrupted_data)
    W=calcW(model).to(device)
    loss = myError(output,clean_data,lamda,W) 
    #+ beta*sparse_loss(rho, corrupted_data).to(device)
        # ===================backward====================
    optimizer.zero_grad()
    loss.backward()
    e.append(loss.item())
    with torch.no_grad():
        optimizer.step()
    # ===================log========================
  with torch.no_grad():
      v=[] #v contains the validation loss of all testset batches
      model_children=list(model.children())
      for j in dl_test:
        corrupted_data_test,clean_data_test=j[:,0,:],j[:,1,:]
        corrupted_data_test=corrupted_data_test.reshape(-1,input_size).to(device)
        clean_data_test=clean_data_test.reshape(-1,input_size).to(device)
        
        
        s=clean_data_test.cpu().numpy()
        clean_data_test[np.where(s.std(axis=1)<delta)]=torch.zeros(clean_data_test[np.where(s.std(axis=1)<delta)].shape,dtype=torch.float64).to(device)
        
        output_test = model(corrupted_data_test)
        W=calcW(model).to(device)
        
        validation = myError(output_test,clean_data_test,lamda,W) 
        #+beta*sparse_loss(rho, corrupted_data_test).to(device)
        v.append(validation.item())
  

  if epoch==0:
        model_old=model
        validation_old=np.array(v).mean()
        validation_new=np.array(v).mean()
        model,validation_old=choose_model(model_old,model,validation_old,validation_new)
        model_old=model
        
        Error.append(np.array(e).mean())
        Validation.append(validation_old)
  else:
    validation_new=np.array(v).mean()
    model,validation_old=choose_model(model_old,model,validation_old,validation_new)
    model_old=model
    
    Error.append(np.array(e).mean())
    Validation.append(validation_old)
    
    
  print('epoch [{}/{}], loss:{:.6f} , validation loss:{:.6f}'
          .format(epoch + 1, Epochs, np.array(e).mean(),validation_old) )


torch.save(model,'AutoencoderNikaia')

In [ ]:
plt.title('Errors')
plt.plot(Error,label = "Training error")
plt.plot(Validation,label = "Validation error")
plt.legend()
plt.show()
#plt.plot(Validation)

In [ ]:
plt.title("Training error")
plt.plot(Error)
plt.show()

In [ ]:
plt.title("Validation error")
plt.plot(Validation)
plt.show()

# Denoising Visualization


In [ ]:
modelPatra=torch.load('AutoencoderPatra')
modelNikaia=torch.load('AutoencoderNikaia')
model=torch.load('AutoencoderWhole')

In [ ]:
j='Nikaia'
#j='Patra'
for i in range(1,3000):
    try:
        samplerate, data = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_R_scaled')
        
        out=clean_signal(data.reshape(-1),modelNikaia,window_length,device)
        
        k=min([data.shape[0],out.shape[0]])
        
        
        res=data[:k]-out[:k]
        

        (mu, sigma) = norm.fit(res)
        plt.title(str(i))
        n, bins, patches = plt.hist(res, 300, facecolor='green',density =True,stacked =True)

        y = scipy.stats.norm.pdf( bins, mu, sigma)
        
        l = plt.plot(bins, y, linewidth=2)
        
        plt.show()
        
        
    except:
        continue

In [ ]:
j='Nikaia'
#j='Patra'
for i in range(2000,3000):
    try:
        samplerate, data = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_R_scaled')
        
        out=clean_signal(data.reshape(-1),modelNikaia,window_length,device)
        
        k=min([data.shape[0],out.shape[0]])
        
        #plt.plot(data)
        #plt.plot(out)
        res=data[:k]-out[:k]
        plt.plot(res)
        plt.show()
        
        
    except:
        continue

# Experiments

In [ ]:
#Experiments with extented spectral gating

j='Patra'
i=43

sr, mixtureR1 = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_R_scaled')
sr, mixtureL1 = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_L_scaled')

s_R1=clean_signal(mixtureR1.reshape(-1),modelPatra,window_length,device)
s_L1=clean_signal(mixtureL1.reshape(-1),modelPatra,window_length,device)


wavfile.write(str(i)+'_R_AEdenoised', sr, s_R1)
wavfile.write(str(i)+'_L_AEdenoised', sr, s_L1)
wavfile.write(str(i)+'_L_AEnoise', sr, mixtureL1-s_L1)
wavfile.write(str(i)+'_R_AEnoise', sr, mixtureR1-s_R1)

In [ ]:
j='Nikaia'
i=2457

sr, mixtureR2 = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_R_scaled')
sr, mixtureL2 = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_L_scaled')
s_R2=clean_signal(mixtureR2.reshape(-1),modelNikaia,window_length,device)
s_L2=clean_signal(mixtureL2.reshape(-1),modelNikaia,window_length,device)


wavfile.write(str(i)+'_R_AEdenoised', sr, s_R2)
wavfile.write(str(i)+'_L_AEdenoised', sr, s_L2)
wavfile.write(str(i)+'_L_AEnoise', sr, mixtureL2-s_L2)
wavfile.write(str(i)+'_R_AEnoise', sr, mixtureR2-s_R2)

In [ ]:
#visualization of  extented spectral denoising
data=mixtureL1
out=s_L1
name='Patra:43'

gridsize = (3, 3)
fig = plt.figure(figsize=(20, 15))
fig.suptitle(name,fontsize=16)
ax1 = plt.subplot2grid(gridsize, (0, 0))
ax2 = plt.subplot2grid(gridsize, (0, 1))
ax3 = plt.subplot2grid(gridsize, (0, 2))

ax4 = plt.subplot2grid(gridsize, (1, 0))
ax5 = plt.subplot2grid(gridsize, (1, 1))
ax6 = plt.subplot2grid(gridsize, (1, 2))


ax1.set_title('Noisy signal:')
ax1.plot(data)
ax2.set_title('Clean signal:')
ax2.plot(out)
ax3.set_title('Residual:')
ax3.plot((data-out))


ax4.specgram(data,NFFT=1024,Fs=48000,noverlap=512)
ax5.specgram(out,NFFT=1024,Fs=48000,noverlap=512)
ax6.specgram(data-out,NFFT=1024,Fs=48000,noverlap=512)
plt.show()

In [ ]:
data=mixtureL2
out=s_L2
name='Nikaia:2457'

gridsize = (3, 3)
fig = plt.figure(figsize=(20, 15))
fig.suptitle(name,fontsize=16)
ax1 = plt.subplot2grid(gridsize, (0, 0))
ax2 = plt.subplot2grid(gridsize, (0, 1))
ax3 = plt.subplot2grid(gridsize, (0, 2))

ax4 = plt.subplot2grid(gridsize, (1, 0))
ax5 = plt.subplot2grid(gridsize, (1, 1))
ax6 = plt.subplot2grid(gridsize, (1, 2))


ax1.set_title('Noisy signal:')
ax1.plot(data)
ax2.set_title('Clean signal:')
ax2.plot(out)
ax3.set_title('Residual:')
ax3.plot((data-out))


ax4.specgram(data,NFFT=1024,Fs=48000,noverlap=512)
ax5.specgram(out,NFFT=1024,Fs=48000,noverlap=512)
ax6.specgram(data-out,NFFT=1024,Fs=48000,noverlap=512)
plt.show()

In [ ]:
#Noise distribution
noisePatra=mixtureL1-s_L1
noiseNikaia=mixtureL2-s_L2

fig, (ax5,ax6)= plt.subplots(2,1,constrained_layout = True)

ax5.title.set_text('Patra noise distribution:'+str(43))
ax6.title.set_text('Nikaia noise distribution:'+str(2457))

ax5.hist( noisePatra,bins=200,range=[-.4,.4])
ax6.hist( noiseNikaia,bins=200,range=[-1.5,1.5])
plt.show()

In [ ]:
#Noise distributions

s_R4=noise_reduce(mixtureR1,sr,1,beta=0.01,time_ms=1)
s_L4=noise_reduce(mixtureL1,sr,1,beta=0.01,time_ms=1)


s_R5=noise_reduce(mixtureR2,sr,1,beta=0.01,time_ms=1)
s_L5=noise_reduce(mixtureL2,sr,1,beta=0.01,time_ms=1)


gridsize = (4, 4)
fig = plt.figure(figsize=(20, 15))


ax1 = plt.subplot2grid(gridsize, (0, 0))
ax2 = plt.subplot2grid(gridsize, (0, 1))
ax3 = plt.subplot2grid(gridsize, (0, 2))
ax4 = plt.subplot2grid(gridsize, (0, 3))



ax5 = plt.subplot2grid(gridsize, (1, 0))
ax6 = plt.subplot2grid(gridsize, (1, 1))
ax7 = plt.subplot2grid(gridsize, (1, 2))
ax8 = plt.subplot2grid(gridsize, (1, 3))




ax1.set_title('Spectral Subtraction noise:43')
ax1.plot((s_L4-mixtureL1))

ax2.set_title('Denoising Autoencoder noise:43')
ax2.plot((s_L1-mixtureL1))

ax3.set_title('Spectral Subtraction noise:2457')
ax3.plot((s_L5-mixtureL2))
ax3.set_xticks(np.arange(0, s_L5.shape[0], step=200000))
ax4.set_title('Denoising Autoencoder noise:2457')
ax4.plot((s_L2-mixtureL2))
ax4.set_xticks(np.arange(0, s_L2.shape[0], step=200000))

ax5.set_title('Noise distribution:')
ax5.hist((s_L4-mixtureL1),200,range=[-0.4,0.4],density=True)
bins5,y5=pdf(s_L4-mixtureL1,200)
ax5.plot(bins5, y5, linewidth=2)

ax6.set_title('Noise distribution:')
ax6.hist((s_L1-mixtureL1),200,range=[-0.4,0.4],density=True)
bins6,y6=pdf((s_L1-mixtureL1),200)
ax6.set_xlim([-0.4,0.4])
ax6.plot(bins6, y6, linewidth=2)

ax7.set_title('Noise distribution:')
ax7.hist((s_L5-mixtureL2),200,range=[-1.,1.],density=True)
bins7,y7=pdf((s_L5-mixtureL2),200)
ax7.plot(bins7, y7, linewidth=2)
ax7.set_xlim([-1.,1.])


ax8.set_title('Noise distribution:')
ax8.hist((s_L2-mixtureL2),200,range=[-1.,1.],density=True)
bins8,y8=pdf((s_L2-mixtureL2),200)
ax8.plot(bins8, y8, linewidth=2)
ax8.set_xlim([-1.,1.])
plt.show()

In [ ]:
#Signals with powerfull noise.
j='Patra'
i=64


sr, mixtureR3 = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_R_scaled')
sr, mixtureL3 = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_L_scaled')

s_R3=clean_signal(mixtureR3.reshape(-1),modelPatra,window_length,device)
s_L3=clean_signal(mixtureL3.reshape(-1),modelPatra,window_length,device)

wavfile.write(str(i)+'_R_AEdenoised', sr, s_R3)
wavfile.write(str(i)+'_L_AEdenoised', sr, s_L3)
wavfile.write(str(i)+'_L_AEnoise', sr, mixtureL3-s_L3)
wavfile.write(str(i)+'_R_AEnoise', sr, mixtureR3-s_R3)

#visualization of denoising
#------------------------------------------------------------------------
data=mixtureL3
out=s_L3
name='Patra:64'

gridsize = (3, 3)
fig = plt.figure(figsize=(20, 15))
fig.suptitle(name,fontsize=16)
ax1 = plt.subplot2grid(gridsize, (0, 0))
ax2 = plt.subplot2grid(gridsize, (0, 1))
ax3 = plt.subplot2grid(gridsize, (0, 2))

ax4 = plt.subplot2grid(gridsize, (1, 0))
ax5 = plt.subplot2grid(gridsize, (1, 1))
ax6 = plt.subplot2grid(gridsize, (1, 2))


ax1.set_title('Noisy signal:')
ax1.plot(data)
ax2.set_title('Clean signal:')
ax2.plot(out)
ax3.set_title('Residual:')
ax3.plot((data-out))


ax4.specgram(data,NFFT=1024,Fs=48000,noverlap=512)
ax5.specgram(out,NFFT=1024,Fs=48000,noverlap=512)
ax6.specgram(data-out,NFFT=1024,Fs=48000,noverlap=512)
plt.show()

# Data denoising

In [ ]:
j='Patra'

modelPatra=torch.load('AutoencoderPatra')

for i in range(1,82):
    try:
        samplerate, data_R = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_R_scaled')
        samplerate, data_L = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_L_scaled')
        out_R=clean_signal(data_R.reshape(-1),modelPatra,window_length,device)
        out_L=clean_signal(data_L.reshape(-1),modelPatra,window_length,device)
        wavfile.write(str(i)+'_pydeNoised_R.wav', samplerate, out_R)
        wavfile.write(str(i)+'_pydeNoised_L.wav', samplerate, out_L)
    except:
        continue

In [ ]:
j='Nikaia'

modelNikaia=torch.load('AutoencoderNikaia')

for i in range(1000,3000):
    try:
        samplerate, data_R = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_R_scaled')
        samplerate, data_L = wavfile.read('/home/myron/Desktop/Diplomatikh/data/F_Covid19/'+j+'_LR_st/'+str(i)+'_L_scaled')
        out_R=clean_signal(data_R.reshape(-1),modelNikaia,window_length,device)
        out_L=clean_signal(data_L.reshape(-1),modelNikaia,window_length,device)
        wavfile.write(str(i)+'_pydeNoised_R.wav', samplerate, out_R)
        wavfile.write(str(i)+'_pydeNoised_L.wav', samplerate, out_L)
    except:
        continue